In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
import math
from functions import split_dataset_LSTM, printHyperparams
%matplotlib inline

In [ ]:
df = pd.read_csv("datasets/nasdaq2007_17.csv", header=None, sep='\t')
print("Number of rows and columns:", df.shape)
# df.head(5)

time_array = np.array([x for x in range(len(df.columns))])
df2 = pd.DataFrame(time_array)

N = 1  # number of time series from dataset
training_percent = 0.8
look_back = 60
sc = MinMaxScaler(feature_range=(0, 1))

# hyperparameters: layers, units, epochs, batch size
hyperparams = []
hyperparams.append([4, 50, 10, 1])
hyperparams.append([4, 50, 10, 16])
hyperparams.append([4, 50, 10, 32])
hyperparams.append([4, 50, 10, 64])
hyperparams.append([4, 50, 10, 128])

f = open("./hyperparams/forecast_table.csv", "w")
# write headers
f.write("layers\tunits\tepochs\tbatch_size\tloss\tval_loss\n")
# close file
f.close()

In [ ]:
for hp in hyperparams:
    # Creating a data structure with 'look_back' time-steps and 1 output
    X_train, y_train, X_test, train_size = split_dataset_LSTM(
        df, 0, training_percent, sc, look_back)

    model = printHyperparams(X_train, y_train, hp[0], hp[1], hp[2], hp[3])

    # Getting the predicted stock price (trained with 1 timeseries)
    predicted_stock_price = model.predict(X_test)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)

    plt.figure(figsize=(14, 7))
    plt.plot(df2.iloc[train_size:].values, df.iloc[0, train_size:].values,
             color='red', label='Real Stock Price')
    plt.plot(df2.iloc[train_size:].values, predicted_stock_price,
             color='blue', label='Predicted Stock Price')
    plt.title('Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.show()